In [6]:
import optuna
import os
from dotenv import load_dotenv
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb
import optuna
import pickle
import os
from dotenv import load_dotenv
import urllib
from pathlib import Path
import duckdb
import pandas as pd
# Load environment variables from .env file

mes_train = [202103, 202104, 202105]
mes_test = [202106]
n_envios = 10
intentos = 100
sampling = 1
study_name = "experimento_sampling_20perc"
#%%

# Print the values to verify
print("mes_train:", mes_train)
print("mes_test:", mes_test)
print("n_envios:", n_envios)
print("sampling:", sampling)
print("study_name:", study_name)

# parametros a setear
dataset_path = 'compe_02'
boost_rounds = 10000
optimizar = True
min_envios = 8000
max_envios = 14000
paso_envios = (max_envios - min_envios) / n_envios

#

# params_objetivo = {
#     'num_leaves' : trial.suggest_int('num_leaves', 20, 1000),
#     'learning_rate' : trial.suggest_float('learning_rate', 0.005, 0.4), # mas bajo, más iteraciones necesita
#     'min_data_in_leaf' : trial.suggest_int('min_data_in_leaf', 1, 1000),
#     'feature_fraction' : trial.suggest_float('feature_fraction', 0.1, .8),
#     'feature_fraction_bynode' : trial.suggest_float('feature_fraction_bynode', 0.1, .8), 
#     'bagging_fraction' : trial.suggest_float('bagging_fraction', 0.05, .08),
#     'drop_rate': trial.suggest_float('drop_rate', 0.1, 3.0),
# }
    

fixed_params = {
    'objective': 'binary',
    'metric': 'custom',
    'boosting_type': 'gbdt',
    'first_metric_only': True,
    'boost_from_average': True,
    'feature_pre_filter': False,
    'force_col_wise' : True,
    'verbose': -1}

#%%
# variables ambiente fijas
load_dotenv()
semillas = os.getenv("semillas")
semillas = [int(x) for x in semillas.split(",")]
semillas = semillas + [x + 1 for x in semillas]
ganancia_acierto = 273000
costo_estimulo = 7000
base_path = ''
modelos_path = base_path + 'modelos/'
db_path = base_path + 'db/'
storage_name = "mysql+mysqldb://{u}:{p}@{ip}:3306/optuna_rf_db".format(p=urllib.parse.quote_plus(os.getenv("password")), u = os.getenv("usersrv"), ip = os.getenv("ip"))

# %%


mes_train: [202103, 202104, 202105]
mes_test: [202106]
n_envios: 10
sampling: 1
study_name: experimento_sampling_20perc


In [2]:


def preparar_data(dbname, mes_train, mes_test, drop_cols=None, sampling = 1):

    print('Preparando data...')

    root_path = "./"

    # Connect to DuckDB database
    con = duckdb.connect(root_path+'duckdb/'+ dbname + ".duckdb")       

    meses = mes_train + mes_test
    
    meses = ', '.join(map(str, meses))

    
    query = (f"""
            SELECT *
            FROM {dbname}
            JOIN {dbname}_lags
            ON {dbname}.foto_mes = {dbname}_lags.foto_mes
            AND {dbname}.numero_de_cliente = {dbname}_lags.numero_de_cliente
            AND {dbname}.clase_ternaria = {dbname}_lags.clase_ternaria
            WHERE {dbname}.foto_mes IN ({meses});""")

    # Execute the join query and fetch the result as a Pandas DataFrame
    data = con.execute(query).fetchdf()

    con.close()

    data = data.drop(['numero_de_cliente_1', 'foto_mes_1', 'clase_ternaria_1'], axis=1)

    # data = pd.read_parquet(dataset_path + dataset_file)

    data['clase_peso'] = 1.0

    data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
    data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

    data['clase_binaria1'] = 0
    data['clase_binaria2'] = 0
    data['clase_binaria1'] = np.where(data['clase_ternaria'] == 'BAJA+2', 1, 0)
    data['clase_binaria2'] = np.where(data['clase_ternaria'] == 'CONTINUA', 0, 1)

#   data = data.loc[data['foto_mes'].isin(mes_train + mes_test)]

    data_continua = data[data['clase_ternaria'] == 'CONTINUA']

    data_continua = data_continua.groupby('foto_mes').sample(frac=sampling, random_state=42)

    data = pd.concat([data_continua, data[data['clase_ternaria'] != 'CONTINUA']])

    print(data.shape)

    print(data.head())

    del data_continua

    train_data = data.loc[data['foto_mes'].isin(mes_train)]
    test_data = data[data['foto_mes'].isin(mes_test)]

    X_train = train_data.drop(['numero_de_cliente','clase_ternaria', 'clase_peso', 'clase_binaria1','clase_binaria2'], axis=1)
    y_train_binaria1 = train_data['clase_binaria1']
    y_train_binaria2 = train_data['clase_binaria2']
    w_train = train_data['clase_peso']

    X_test = test_data.drop(['numero_de_cliente','clase_ternaria', 'clase_peso', 'clase_binaria1','clase_binaria2'], axis=1)
    y_test_binaria1 = test_data['clase_binaria1']
    y_test_class = test_data['clase_ternaria']
    w_test = test_data['clase_peso']

    print('Data preparada')

    return X_train, y_train_binaria1, y_train_binaria2, w_train, X_test, y_test_class, y_test_binaria1, w_test



In [7]:

X_train, y_train_binaria1, y_train_binaria2, w_train, X_test, y_test_class, y_test_binaria1, w_test = preparar_data(
                                                                                                                    dbname=dataset_path,
                                                                                                                    mes_train= mes_train,
                                                                                                                    mes_test= mes_test,
                                                                                                                    sampling= 0.05)



Preparando data...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

(40705, 702)
        numero_de_cliente  foto_mes  active_quarter  cliente_vip  internet  \
626617         1556250865    202103               1            0         0   
269385          527108152    202103               1            0         0   
119493          812197332    202103               1            0         0   
124533          275912629    202103               1            0         0   
498128          606843249    202103               1            0         0   

        cliente_edad  cliente_antiguedad  mrentabilidad  mrentabilidad_annual  \
626617            31                   7         235.85               3131.62   
269385            45                 167        1803.55              19898.90   
119493            54                 154       -1959.17               3929.57   
124533            69                 322         714.90              21442.32   
498128            44                 159         247.09             -10020.90   

        mcomisiones  ...  lag2_

In [9]:
X_train = X_train.drop(['foto_mes'], axis=1)

X_train.columns

Index(['active_quarter', 'cliente_vip', 'internet', 'cliente_edad',
       'cliente_antiguedad', 'mrentabilidad', 'mrentabilidad_annual',
       'mcomisiones', 'mactivos_margen', 'mpasivos_margen',
       ...
       'lag1_tc_proporcion_consumo_total_limite_total_cubierto',
       'lag2_tc_proporcion_consumo_total_limite_total_cubierto',
       'lag1_tc_proporcion_pago_pesos', 'lag2_tc_proporcion_pago_pesos',
       'lag1_tc_proporcion_pago_dolares', 'lag2_tc_proporcion_pago_dolares',
       'lag1_tc_proporcion_adelanto_pesos',
       'lag2_tc_proporcion_adelanto_pesos',
       'lag1_tc_proporcion_adelanto_dolares',
       'lag2_tc_proporcion_adelanto_dolares'],
      dtype='object', length=696)

In [10]:
X_train.shape[1]

696

In [14]:
[f'trial.suggest_int(mono_"{X_train.columns[x]}",-1,1)' for x in np.arange(0,X_train.shape[1])]

['trial.suggest_int("active_quarter",-1,1)',
 'trial.suggest_int("cliente_vip",-1,1)',
 'trial.suggest_int("internet",-1,1)',
 'trial.suggest_int("cliente_edad",-1,1)',
 'trial.suggest_int("cliente_antiguedad",-1,1)',
 'trial.suggest_int("mrentabilidad",-1,1)',
 'trial.suggest_int("mrentabilidad_annual",-1,1)',
 'trial.suggest_int("mcomisiones",-1,1)',
 'trial.suggest_int("mactivos_margen",-1,1)',
 'trial.suggest_int("mpasivos_margen",-1,1)',
 'trial.suggest_int("cproductos",-1,1)',
 'trial.suggest_int("tcuentas",-1,1)',
 'trial.suggest_int("ccuenta_corriente",-1,1)',
 'trial.suggest_int("mcuenta_corriente_adicional",-1,1)',
 'trial.suggest_int("mcuenta_corriente",-1,1)',
 'trial.suggest_int("ccaja_ahorro",-1,1)',
 'trial.suggest_int("mcaja_ahorro",-1,1)',
 'trial.suggest_int("mcaja_ahorro_adicional",-1,1)',
 'trial.suggest_int("mcaja_ahorro_dolares",-1,1)',
 'trial.suggest_int("cdescubierto_preacordado",-1,1)',
 'trial.suggest_int("mcuentas_saldo",-1,1)',
 'trial.suggest_int("ctarjeta